In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session

session = get_active_session()

st.markdown("## 🔍 Check What's Actually in Snowflake Table")

# Query to check the actual data in Snowflake
check_sql = """
SELECT 
    primary_ticker,
    event_timestamp,
    LENGTH(transcript::STRING) AS transcript_json_length,
    transcript
FROM unique_transcripts
LIMIT 1
"""

if st.button("🔍 Check Snowflake Data", use_container_width=True):
    with st.spinner("Querying Snowflake table..."):
        try:
            result = session.sql(check_sql).collect()
            
            if result:
                row = result[0]
                ticker = row['PRIMARY_TICKER']
                json_length = row['TRANSCRIPT_JSON_LENGTH']
                transcript_json = row['TRANSCRIPT']
                
                st.success(f"✅ Found data for {ticker}")
                st.metric("JSON Length in Snowflake", f"{json_length:,} characters")
                
                # Try to parse the JSON
                import json as py_json
                try:
                    if isinstance(transcript_json, str):
                        parsed = py_json.loads(transcript_json)
                    else:
                        parsed = transcript_json
                    
                    num_entries = len(parsed.get('parsed_transcript', []))
                    num_speakers = len(parsed.get('speaker_mapping', []))
                    
                    st.success(f"✅ JSON is valid")
                    col1, col2 = st.columns(2)
                    col1.metric("Transcript Entries", num_entries)
                    col2.metric("Speakers", num_speakers)
                    
                    # Show speaker breakdown
                    st.subheader("Speaker Breakdown:")
                    for speaker in parsed.get('speaker_mapping', []):
                        name = speaker['speaker_data'].get('name', 'Unknown')
                        role = speaker['speaker_data'].get('role', 'N/A')
                        st.write(f"- {name} ({role})")
                    
                    # Show first 3 entries
                    st.subheader("First 3 Transcript Entries:")
                    for i, entry in enumerate(parsed.get('parsed_transcript', [])[:3]):
                        speaker_id = entry['speaker']
                        text = entry['text'][:200] + "..." if len(entry['text']) > 200 else entry['text']
                        st.text_area(f"Entry {i+1} (Speaker {speaker_id})", value=text, height=100)
                    
                    # Show last 3 entries to check if Q&A is there
                    st.subheader("Last 3 Transcript Entries (should be Q&A):")
                    for i, entry in enumerate(parsed.get('parsed_transcript', [])[-3:]):
                        speaker_id = entry['speaker']
                        text = entry['text'][:200] + "..." if len(entry['text']) > 200 else entry['text']
                        # Find speaker name
                        speaker_info = next((s for s in parsed.get('speaker_mapping', []) if s['speaker'] == speaker_id), None)
                        speaker_name = speaker_info['speaker_data'].get('name', 'Unknown') if speaker_info else 'Unknown'
                        speaker_role = speaker_info['speaker_data'].get('role', '') if speaker_info else ''
                        
                        st.text_area(f"{speaker_name} ({speaker_role})", value=text, height=100)
                    
                    # Compare with expected CSV length
                    st.markdown("---")
                    st.subheader("Comparison with CSV:")
                    st.write("**Expected from CSV**: ~16,629 characters with 31 entries")
                    st.write(f"**Actual in Snowflake**: {json_length:,} characters with {num_entries} entries")
                    
                    if num_entries < 31:
                        st.error(f"⚠️ DATA TRUNCATED! Missing {31 - num_entries} entries")
                    elif json_length < 15000:
                        st.error(f"⚠️ JSON appears truncated! Expected ~16,629 chars")
                    else:
                        st.success("✅ Data appears complete!")
                    
                except Exception as e:
                    st.error(f"❌ Failed to parse JSON: {str(e)}")
                    st.text_area("Raw transcript (first 2000 chars):", 
                               value=str(transcript_json)[:2000],
                               height=300)
            else:
                st.warning("No data found in table")
                
        except Exception as e:
            st.error(f"❌ Error querying table: {str(e)}")
            import traceback
            st.code(traceback.format_exc())
else:
    st.info("👆 Click to check what data is actually in the Snowflake table")


In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session

session = get_active_session()

st.markdown("## 🔄 Reload Enhanced Transcripts")
st.markdown("""
The transcript data has been enhanced to include analyst Q&A sections for all companies.
Click the button below to reload the `unique_transcripts` table with the enhanced data.
""")

if st.button("🔄 Reload Enhanced Transcripts", use_container_width=True):
    with st.spinner("Reloading enhanced transcript data..."):
        try:
            # Upload file to stage
            st.info("Step 1: Uploading enhanced CSV to stage...")
            session.sql("""
                PUT file:///Users/boconnor/fsi-cortex-assistant/dataops/event/DATA/unique_transcripts.csv 
                @CSV_DATA_STAGE 
                auto_compress=false 
                overwrite=true
            """).collect()
            
            # Backup existing data
            st.info("Step 2: Creating backup...")
            session.sql("""
                CREATE OR REPLACE TABLE unique_transcripts_backup AS 
                SELECT * FROM unique_transcripts
            """).collect()
            
            # Truncate table
            st.info("Step 3: Truncating table...")
            session.sql("TRUNCATE TABLE unique_transcripts").collect()
            
            # Reload data
            st.info("Step 4: Loading enhanced data...")
            result = session.sql("""
                COPY INTO unique_transcripts
                FROM @CSV_DATA_STAGE/unique_transcripts.csv
                FILE_FORMAT = (FORMAT_NAME = 'CSV_FORMAT')
                ON_ERROR = 'CONTINUE'
            """).collect()
            
            rows_loaded = result[0]['rows_loaded'] if result else 0
            
            # Verify
            st.info("Step 5: Verifying data...")
            verify_df = session.sql("""
                SELECT 
                    COUNT(*) as total_transcripts,
                    COUNT(DISTINCT primary_ticker) as unique_tickers
                FROM unique_transcripts
            """).to_pandas()
            
            st.success(f"✅ Successfully loaded {rows_loaded} rows!")
            
            col1, col2 = st.columns(2)
            col1.metric("Total Transcripts", verify_df['TOTAL_TRANSCRIPTS'].iloc[0])
            col2.metric("Unique Tickers", verify_df['UNIQUE_TICKERS'].iloc[0])
            
            # Check analyst coverage
            st.subheader("Analyst Coverage Check")
            analyst_check = session.sql("""
                WITH analyst_counts AS (
                    SELECT 
                        primary_ticker,
                        ARRAY_SIZE(PARSE_JSON(transcript):parsed_transcript) as total_entries,
                        ARRAY_SIZE(ARRAY_AGG(s.value) 
                            FILTER (WHERE s.value:speaker_data.role = 'Analyst')) as analyst_count
                    FROM unique_transcripts,
                    LATERAL FLATTEN(input => PARSE_JSON(transcript):speaker_mapping) s
                    GROUP BY primary_ticker, transcript
                )
                SELECT 
                    COUNT(*) as transcripts_with_analysts,
                    MIN(analyst_count) as min_analysts,
                    MAX(analyst_count) as max_analysts,
                    ROUND(AVG(analyst_count), 1) as avg_analysts
                FROM analyst_counts
                WHERE analyst_count > 0
            """).to_pandas()
            
            col1, col2, col3, col4 = st.columns(4)
            col1.metric("Transcripts with Analysts", analyst_check['TRANSCRIPTS_WITH_ANALYSTS'].iloc[0])
            col2.metric("Min Analysts", analyst_check['MIN_ANALYSTS'].iloc[0])
            col3.metric("Max Analysts", analyst_check['MAX_ANALYSTS'].iloc[0])
            col4.metric("Avg Analysts", analyst_check['AVG_ANALYSTS'].iloc[0])
            
            if analyst_check['TRANSCRIPTS_WITH_ANALYSTS'].iloc[0] == 92:
                st.success("✅ All 92 transcripts now have analyst Q&A sections!")
            
        except Exception as e:
            st.error(f"❌ Error reloading data: {str(e)}")
            import traceback
            st.code(traceback.format_exc())
else:
    st.info("👆 Click the button above to reload the enhanced transcript data")


# Cortex for Financial Services
Snowflake has been the trusted data platform for our financial services customers, and we are excited to show case all our latest AI functionality to further supercharge your workflow. In this notebook, we will walk through:

1. Leverage Snowflake marketplace to access market data
2. Leverage Snowflake Cortex AISQL to process the unstructured data
3. Leverage Snowflake Data Science Agent to accelerate ML model building

In [ ]:
import streamlit as st
import pandas as pd
import time
import random
from datetime import datetime, timedelta
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col

# --- Initialize Session and Ticker List ---
# Get the active Snowpark session
session = get_active_session()

# This holds the list of tickers. It fetches a live list from Snowflake on the first run.
if 'tickers_list' not in st.session_state:
    with st.spinner("Fetching initial ticker list from Snowflake..."):
        ticker_df = session.table('ai_transcripts_analysts_sentiments').select('PRIMARY_TICKER').distinct().limit(10)
        st.session_state.tickers_list = [row['PRIMARY_TICKER'] for row in ticker_df.collect()]

# --- Custom CSS for styling ---
st.markdown("""
    <style>
        /* Background is now white (Streamlit default) */
        .stTextArea, .stCodeBlock, .stDataFrame {
            background-color: #F0F2F6; /* Light gray for contrast on white */
            border: 1px solid #D1D9E1; /* Subtle border */
        }
        h1, h2, h3 {
            color: #001f3f !important; /* Navy Blue */
        }
        .block-container {
            padding-top: 2rem;
        }
        .stButton>button {
            border-color: #29B5E8; /* Snowflake blue */
            background-color: #29B5E8;
            color: white;
            margin-bottom: 2rem; /* Add space after the button */
        }
        .stButton>button:hover {
            border-color: #1F8CBF; /* Darker blue */
            background-color: #1F8CBF;
        }
        /* Style for the metric value */
        [data-testid="stMetricValue"] {
            font-size: 2.5rem;
        }
    </style>
""", unsafe_allow_html=True)

# --- AISQL Query Definition ---
aisql_query = """-- Create analyst sentiment table from raw transcripts
CREATE OR REPLACE TABLE ai_transcripts_analysts_sentiments AS (
    WITH
    ai_analysis AS (
        SELECT
            primary_ticker,
            event_timestamp,
            transcript,
            ai_complete(
                'claude-4-sonnet',
                CONCAT_WS('\\n',
                    'You are analyzing sell side and buy side analysts sentiment in a public company''s earnings call transcript.',
                    'Focus ONLY on analyst questions, tone, and reactions to management responses.- Ignore management’s prepared remarks unless directly referenced by analysts. - Pay attention to how analysts compare results to prior earnings calls and to market expectations.',
                    'Use a CONSISTENT STANDARD across all earnings calls, regardless of sector, size, or company specifics.',
                    'Evaluate (internally, equal weight) the following aspects:',
                    'a) Guidance/Financial Outlook (revenue, earnings, margins, cash flow)',
                    'b) Strategy/Product/Innovation (new initiatives, products, technologies)',
                    'c) Competitive/Market Positioning (market share, competition, macro trends)',
                    'd) Management Execution & Credibility (track record, transparency, consistency)',
                    'Compute ONE overall analyst sentiment score on a 1-10 scale, where:',
                    '1 = Extremely pessimistic, 5 = Neutral/mixed, 10 = Extremely optimistic.',
                    'e) create a column with wonderful emojis that reflect the sentiment score',
                    'Return your response as valid JSON with this exact format:',
                    '{"score": <integer 1-10>, "reason": <brief explanation covering the four aspects>, "analyst_count": <integer number of unique analysts>}',
                    'Transcript:',
                    transcript
                )
            ) AS ai_response
        FROM unique_transcripts
    )

    SELECT
        primary_ticker,
        event_timestamp,
        (TRY_PARSE_JSON(ai_response):score)::INT AS sentiment_score,
        (TRY_PARSE_JSON(ai_response):analyst_count)::INT AS unique_analyst_count,
        (TRY_PARSE_JSON(ai_response):reason)::STRING AS sentiment_reason
    FROM ai_analysis
);"""

# --- Fetch Real Transcripts from Snowflake ---
def fetch_transcripts_from_snowflake():
    """
    Fetches actual transcripts from the UNIQUE_TRANSCRIPTS table in Snowflake
    and converts the JSON format into readable text for preview.
    """
    import json
    
    # Query the UNIQUE_TRANSCRIPTS table
    transcripts_df = session.table('UNIQUE_TRANSCRIPTS').select(
        'PRIMARY_TICKER', 'EVENT_TIMESTAMP', 'TRANSCRIPT'
    ).limit(5).to_pandas()
    
    # Parse the JSON transcripts to extract readable text
    def parse_transcript_json(transcript_json_str):
        """Parse the JSON transcript and convert to readable text format."""
        try:
            transcript_data = json.loads(transcript_json_str)
            parsed_transcript = transcript_data.get('parsed_transcript', [])
            speaker_mapping = {s['speaker']: s['speaker_data'] for s in transcript_data.get('speaker_mapping', [])}
            
            # Build readable transcript
            text_parts = []
            for entry in parsed_transcript[:15]:  # Limit to first 15 exchanges for preview
                speaker_id = entry.get('speaker')
                text = entry.get('text', '')
                speaker_info = speaker_mapping.get(speaker_id, {})
                speaker_name = speaker_info.get('name', f'Speaker {speaker_id}')
                speaker_role = speaker_info.get('role', '')
                
                if speaker_role:
                    text_parts.append(f"{speaker_name} ({speaker_role}): {text}")
                else:
                    text_parts.append(f"{speaker_name}: {text}")
            
            return '\n\n'.join(text_parts) + '\n\n[... transcript continues ...]'
        except Exception as e:
            return str(transcript_json_str)[:500]  # Fallback to raw text preview
    
    # Convert JSON transcripts to readable format
    transcripts_df['TRANSCRIPT_TEXT'] = transcripts_df['TRANSCRIPT'].apply(parse_transcript_json)
    
    return transcripts_df

# --- Dummy Data for Preview (Now using real Snowflake data) ---
made_up_transcripts_list = []

def create_dummy_source_preview():
    """
    Creates a DataFrame showing actual source transcript data from Snowflake.
    """
    try:
        # Fetch real transcripts from Snowflake
        transcripts_df = fetch_transcripts_from_snowflake()
        
        # Create preview dataframe with readable format
        preview_data = {
            'PRIMARY_TICKER': transcripts_df['PRIMARY_TICKER'],
            'EVENT_TIMESTAMP': transcripts_df['EVENT_TIMESTAMP'],
            'TRANSCRIPT': transcripts_df['TRANSCRIPT_TEXT']
        }
        return pd.DataFrame(preview_data)
    except Exception as e:
        st.error(f"Error fetching transcripts: {e}")
        return pd.DataFrame()

# --- Streamlit App UI ---

st.markdown('''## 📝 Cortex AISQL: Turn your Unstructured data into Structured

**AI-based sentiment dataset** for the **Dow Jones 30 earnings calls** (2024+).
- **Source**: annotated Q&A transcripts from `COMPANY_EVENT_TRANSCRIPT_ATTRIBUTES_V2`.
- **AI Scoring**: Claude (`claude-4-sonnet`) analyzed only **analyst questions/tone**, ignoring management’s scripted remarks.
- **Output**: JSON with
  - `sentiment_score` (1–10 scale),
  - `reason` (brief justification),
  - `analyst_count` (unique analysts).

''')

with st.expander("Illustrative Source Data Preview", expanded=True):
    st.markdown("This is a preview of the **source transcript data** for the first 5 tickers. The AISQL query below is run against this type of unstructured data to generate the final analysis. This example is based on transcript structures found in the **Snowflake Public Data for Financial Services**.")
    dummy_df = create_dummy_source_preview()
    st.dataframe(dummy_df, use_container_width=True)

    st.markdown("--- \n **Example Transcript Detail:**")
    if not dummy_df.empty:
        example_transcript = dummy_df.iloc[0]['TRANSCRIPT']
        st.text_area("Full Transcript Example", value=example_transcript, height=400, disabled=True, label_visibility="collapsed")
    else:
        st.info("No transcript data available to display.")


st.subheader("AISQL Data Generation Code")
st.markdown("The data in the `ai_transcripts_analysts_sentiments` table was generated using the following AISQL query, which uses a GenAI model to analyze each transcript.")
st.code(aisql_query, language='sql')

start_button = st.button("❄️ Generate Earnings Call Sentiment Scores", use_container_width=True)

if start_button:
    with st.spinner('Connecting to Snowflake and fetching live data...'):
        # --- REAL STREAMLIT-IN-SNOWFLAKE CODE ---
        # The session is already active from the top of the script.
        
        # Use the list of tickers we already fetched and stored in the session state.
        tickers_list = st.session_state.tickers_list

        # Fetch the full data for those 10 tickers.
        results_df = session.table("ai_transcripts_analysts_sentiments").drop('ai_response') \
                                .filter(col("PRIMARY_TICKER").in_(tickers_list)) \
                                .to_pandas()
    
    # --- ERROR HANDLING: Check if the DataFrame is empty ---
    if not results_df.empty:
        st.subheader("Live Analyst Sentiment Analysis Results")
        st.markdown("Data loaded from the `ai_transcripts_analysts_sentiments` table.")
        
        # --- Dashboard Metrics ---
        avg_score = results_df['SENTIMENT_SCORE'].mean()
        total_analysts = results_df['UNIQUE_ANALYST_COUNT'].sum()
        most_positive_ticker = results_df.loc[results_df['SENTIMENT_SCORE'].idxmax()]['PRIMARY_TICKER']

        col1, col2, col3 = st.columns(3)
        col1.metric("Average Sentiment Score", f"{avg_score:.1f} / 10")
        col2.metric("Total Analysts Covered", f"{total_analysts}")
        col3.metric("Most Positive Ticker", most_positive_ticker)

        # --- Results Table ---
        
        # Function to add emojis based on score
        def score_to_emoji(score):
            if score >= 8:
                return '🚀'
            elif score >= 6:
                return '👍'
            elif score >= 4:
                return '🤔'
            else:
                return '📉'

        # Create the 'EMOJI' column
        results_df['EMOJI'] = results_df['SENTIMENT_SCORE'].apply(score_to_emoji)
        
        # Function to apply the font-size style
        def double_font_size(val):
            return 'font-size: 400%'

        # Apply style to the 'EMOJI' column using .map and display
        st.dataframe(
            results_df.style.map(double_font_size, subset=['EMOJI']),
            use_container_width=True
        )

    else:
        st.warning("No data found in the table for the selected tickers. The table might be empty or still processing.")

else:
    st.info("Click the button above to connect to Snowflake and load the live sentiment data.")


In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session

# Get the active Snowpark session
session = get_active_session()

st.markdown("## 🎯 Create Analyst Sentiment Table")
st.markdown("""
This cell will create the `ai_transcripts_analysts_sentiments` table by analyzing 
earnings call transcripts from the `unique_transcripts` table using Cortex AI.
""")

# Define the SQL query
create_table_sql = """
-- create analyst sentiment for DOW JONES 30 company 
-- Note timestamp is in UTC

CREATE TABLE IF NOT EXISTS ai_transcripts_analysts_sentiments AS (
WITH 
parsed_transcripts AS (
    SELECT
        primary_ticker,
        event_timestamp,
        event_type,
        created_at,
        transcript,
        -- Parse the JSON transcript and extract speaker mapping
        PARSE_JSON(transcript) AS transcript_json
    FROM unique_transcripts
),
formatted_transcripts AS (
    SELECT
        primary_ticker,
        event_timestamp,
        event_type,
        created_at,
        -- Build formatted transcript with speaker labels
        LISTAGG(
            CASE 
                WHEN speaker_data.value:speaker_data.role::STRING IS NOT NULL 
                THEN speaker_data.value:speaker_data.name::STRING || ' (' || speaker_data.value:speaker_data.role::STRING || '): ' || parsed_entry.value:text::STRING
                ELSE speaker_data.value:speaker_data.name::STRING || ': ' || parsed_entry.value:text::STRING
            END,
            '\n\n'
        ) WITHIN GROUP (ORDER BY parsed_entry.index) AS transcript_text
    FROM parsed_transcripts,
    LATERAL FLATTEN(input => transcript_json:parsed_transcript) parsed_entry,
    LATERAL FLATTEN(input => transcript_json:speaker_mapping) speaker_data
    WHERE speaker_data.value:speaker::INT = parsed_entry.value:speaker::INT
    GROUP BY primary_ticker, event_timestamp, event_type, created_at
),
ai_analysis AS (
    SELECT
        primary_ticker,
        event_timestamp,
        event_type,
        created_at,
        transcript_text,
        ai_complete(
            'claude-4-sonnet',
            CONCAT_WS('\\n',
                'You are analyzing sell side and buy side analysts sentiment in a public company''s earnings call transcript.',
                'Focus ONLY on analyst questions, tone, and reactions to management responses. Ignore management''s prepared remarks unless directly referenced by analysts.',
                'Pay attention to how analysts compare results to prior earnings calls and to market expectations.',
                'Use a CONSISTENT STANDARD across all earnings calls, regardless of sector, size, or company specifics.',
                'Evaluate (internally, equal weight) the following aspects:',
                'a) Guidance/Financial Outlook (revenue growth, earnings, margins, bookings, cash flow)',
                'b) Strategy/Product/Innovation (new initiatives, products, technologies, or business models)',
                'c) Competitive/Market Positioning (market share, competition, regulation, macro/sector trends)',
                'd) Management Execution & Credibility (track record, transparency, quality of responses, consistency with prior guidance)',
                'Compute ONE overall analyst sentiment score on a 1-10 scale, where:',
                '1 = Extremely pessimistic/concerned, 5 = Neutral/mixed, 10 = Extremely optimistic/bullish.',
                'e) A wonderful emoji that reflects the sentiment score',
                'Return your response as valid JSON with this exact format:',
                '{"score": <integer 1-10>, "emoji":"<an emoji>","reason": "<brief explanation covering the four aspects>", "analyst_count": <integer number of unique analysts>}',
                '',
                'IMPORTANT: Only analyze the parts where Analysts are speaking. Look for speaker labels like "Analyst" in the transcript below.',
                '',
                'Transcript:',
                transcript_text
            )
        ) AS ai_response
    FROM formatted_transcripts
)

SELECT
    ai_response,
    primary_ticker,
    event_timestamp,
    event_type,
    created_at,
    /* Safely parse JSON. If parsing fails, these will be NULL. */
    (TRY_PARSE_JSON(ai_response):emoji)::TEXT      AS emoji,
    (TRY_PARSE_JSON(ai_response):score)::INT      AS sentiment_score,
    (TRY_PARSE_JSON(ai_response):analyst_count)::INT AS unique_analyst_count,
    (TRY_PARSE_JSON(ai_response):reason)::STRING  AS sentiment_reason
FROM ai_analysis
-- where unique_analyst_count > 1 -- remove data that only contains speakers from the public company
ORDER BY primary_ticker, event_timestamp
);
"""

# Show the SQL
st.code(create_table_sql, language='sql')

# Button to execute
if st.button("🚀 Create/Refresh Sentiment Table", use_container_width=True):
    with st.spinner("Creating table and analyzing transcripts with Cortex AI... This may take several minutes."):
        try:
            # Execute the CREATE TABLE statement
            session.sql(create_table_sql).collect()
            st.success("✅ Table `ai_transcripts_analysts_sentiments` created successfully!")
            
            # Query the results
            results_df = session.sql("SELECT * FROM ai_transcripts_analysts_sentiments").to_pandas()
            
            st.subheader(f"📊 Results: {len(results_df)} rows")
            st.dataframe(results_df, use_container_width=True)
            
        except Exception as e:
            st.error(f"❌ Error creating table: {str(e)}")
else:
    st.info("👆 Click the button above to create the sentiment analysis table.")
    
    # Check if table already exists and show preview
    try:
        existing_count = session.sql("SELECT COUNT(*) as cnt FROM ai_transcripts_analysts_sentiments").collect()
        if existing_count:
            count = existing_count[0]['CNT']
            st.info(f"ℹ️ Table already exists with {count} rows. Click the button to refresh.")
            
            # Show preview
            preview_df = session.sql("SELECT * FROM ai_transcripts_analysts_sentiments LIMIT 10").to_pandas()
            st.dataframe(preview_df, use_container_width=True)
    except:
        st.warning("⚠️ Table does not exist yet. Click the button to create it.")


In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session

session = get_active_session()

st.markdown("## 🔍 Debug Transcript Parsing")
st.markdown("This cell helps debug how transcripts are being parsed.")

# Test query to see what we're getting
debug_sql = """
WITH 
parsed_transcripts AS (
    SELECT
        primary_ticker,
        transcript,
        PARSE_JSON(transcript) AS transcript_json
    FROM unique_transcripts
    LIMIT 1
),
flattened_entries AS (
    SELECT
        primary_ticker,
        parsed_entry.index AS entry_index,
        parsed_entry.value:speaker::INT AS speaker_id,
        parsed_entry.value:text::STRING AS entry_text
    FROM parsed_transcripts,
    LATERAL FLATTEN(input => transcript_json:parsed_transcript) parsed_entry
),
speaker_info AS (
    SELECT
        primary_ticker,
        speaker_data.value:speaker::INT AS speaker_id,
        speaker_data.value:speaker_data.name::STRING AS speaker_name,
        speaker_data.value:speaker_data.role::STRING AS speaker_role
    FROM parsed_transcripts,
    LATERAL FLATTEN(input => transcript_json:speaker_mapping) speaker_data
)
SELECT 
    f.primary_ticker,
    f.entry_index,
    s.speaker_name,
    s.speaker_role,
    LEFT(f.entry_text, 100) AS text_preview,
    LENGTH(f.entry_text) AS text_length
FROM flattened_entries f
JOIN speaker_info s ON f.primary_ticker = s.primary_ticker AND f.speaker_id = s.speaker_id
ORDER BY f.entry_index
"""

st.code(debug_sql, language='sql')

if st.button("🔍 Run Debug Query", use_container_width=True):
    with st.spinner("Analyzing transcript structure..."):
        try:
            result_df = session.sql(debug_sql).to_pandas()
            
            st.success(f"✅ Found {len(result_df)} transcript entries")
            
            # Show summary
            ticker = result_df.iloc[0]['PRIMARY_TICKER'] if len(result_df) > 0 else 'N/A'
            analyst_count = len(result_df[result_df['SPEAKER_ROLE'] == 'Analyst'])
            
            col1, col2, col3 = st.columns(3)
            col1.metric("Ticker", ticker)
            col2.metric("Total Entries", len(result_df))
            col3.metric("Analyst Entries", analyst_count)
            
            # Show the data
            st.dataframe(result_df, use_container_width=True)
            
            # Now test the LISTAGG
            st.subheader("Testing LISTAGG Concatenation")
            
            listagg_sql = """
            WITH 
            parsed_transcripts AS (
                SELECT
                    primary_ticker,
                    PARSE_JSON(transcript) AS transcript_json
                FROM unique_transcripts
                LIMIT 1
            )
            SELECT
                primary_ticker,
                LISTAGG(
                    CASE 
                        WHEN speaker_data.value:speaker_data.role::STRING IS NOT NULL 
                        THEN speaker_data.value:speaker_data.name::STRING || ' (' || speaker_data.value:speaker_data.role::STRING || '): ' || parsed_entry.value:text::STRING
                        ELSE speaker_data.value:speaker_data.name::STRING || ': ' || parsed_entry.value:text::STRING
                    END,
                    '\\n\\n'
                ) WITHIN GROUP (ORDER BY parsed_entry.index) AS transcript_text,
                LENGTH(LISTAGG(
                    CASE 
                        WHEN speaker_data.value:speaker_data.role::STRING IS NOT NULL 
                        THEN speaker_data.value:speaker_data.name::STRING || ' (' || speaker_data.value:speaker_data.role::STRING || '): ' || parsed_entry.value:text::STRING
                        ELSE speaker_data.value:speaker_data.name::STRING || ': ' || parsed_entry.value:text::STRING
                    END,
                    '\\n\\n'
                ) WITHIN GROUP (ORDER BY parsed_entry.index)) AS total_length
            FROM parsed_transcripts,
            LATERAL FLATTEN(input => transcript_json:parsed_transcript) parsed_entry,
            LATERAL FLATTEN(input => transcript_json:speaker_mapping) speaker_data
            WHERE speaker_data.value:speaker::INT = parsed_entry.value:speaker::INT
            GROUP BY primary_ticker
            """
            
            listagg_result = session.sql(listagg_sql).to_pandas()
            
            if len(listagg_result) > 0:
                total_length = listagg_result.iloc[0]['TOTAL_LENGTH']
                transcript_text = listagg_result.iloc[0]['TRANSCRIPT_TEXT']
                
                st.metric("Concatenated Transcript Length", f"{total_length:,} characters")
                
                # Show first 2000 chars
                st.text_area("First 2000 characters of formatted transcript:", 
                           value=transcript_text[:2000], 
                           height=300)
                
                # Show last 2000 chars to see if we have Q&A
                st.text_area("Last 2000 characters of formatted transcript:", 
                           value=transcript_text[-2000:] if len(transcript_text) > 2000 else transcript_text, 
                           height=300)
                
                # Check for analyst mentions
                analyst_mentions = transcript_text.count('(Analyst)')
                st.metric("Number of '(Analyst)' labels found", analyst_mentions)
                
                if analyst_mentions == 0:
                    st.error("⚠️ No analyst labels found in the concatenated transcript!")
                else:
                    st.success(f"✅ Found {analyst_mentions} analyst entries in the transcript")
            
        except Exception as e:
            st.error(f"❌ Error: {str(e)}")
            import traceback
            st.code(traceback.format_exc())
else:
    st.info("👆 Click the button above to debug transcript parsing")


In [ ]:
import streamlit as st
import altair as alt
from snowflake.snowpark.context import get_active_session

# --- Title and Introduction ---
st.title("❄️ Live FSI Data from Snowflake for Financial Services")
st.markdown(
    "This app runs directly in Snowflake and uses the data from the **Snowflake Marketplace**."
)

# --- Get Active Session and Query Data ---
# Get the current session object provided by the Streamlit in Snowflake environment
session = get_active_session()

# This query now calculates the offset needed to make the latest date appear as '2025-10-09'
query = """
    WITH DateOffset AS (
        -- First, calculate the difference in days between the absolute latest date
        -- in the table and our target date of Oct 9, 2025.
        SELECT
            DATEDIFF(
                day,
                MAX(TO_TIMESTAMP_NTZ(DATE, 9)::DATE),
                '2025-10-09'::DATE
            ) AS days_to_add
        FROM SWT_LONDON_2025.FIN_SERV.FSI_DATA
    )
    -- Now, apply this calculated offset to every date in the result set.
    SELECT
        TICKER,
        ASSET_CLASS,
        PRIMARY_EXCHANGE_NAME,
        DATEADD(
            day,
            (SELECT days_to_add FROM DateOffset),
            TO_TIMESTAMP_NTZ(DATE, 9)::DATE
        ) as DATE,
        PRICE,
        RETURN,
        IS_SPLIT,
        Y
    FROM SWT_LONDON_2025.FIN_SERV.FSI_DATA
    ORDER BY DATE DESC
    LIMIT 500;
"""

# Execute the query using the Snowpark session and convert to a Pandas DataFrame
data_frame = session.sql(query).to_pandas()


# --- Display the "Nice Looking" Dataframe ---
st.header("FSI Data Explorer")
st.dataframe(
    data_frame,
    use_container_width=True,
    hide_index=True,
    column_config={
        "TICKER": st.column_config.TextColumn(
            "Ticker Symbol",
            help="The stock ticker symbol.",
            width="small"
        ),
        "ASSET_CLASS": st.column_config.TextColumn(
            "Asset Class",
            width="small"
        ),
        "PRIMARY_EXCHANGE_NAME": "Primary Exchange",
        "DATE": st.column_config.DateColumn(
            "Date",
            format="DD MMM, YYYY" # Format for readability
        ),
        "PRICE": st.column_config.NumberColumn(
            "Price (USD)",
            help="Closing price in US Dollars.",
            format="$%.2f"
        ),
        "RETURN": st.column_config.NumberColumn(
            "Daily Return",
            help="The daily percentage return.",
            format="%.2f%%"
        ),
        "IS_SPLIT": st.column_config.CheckboxColumn(
            "Stock Split?",
            help="Indicates if a stock split occurred.",
            width="small"
        ),
        "Y": st.column_config.ProgressColumn(
            "Target Probability (Y)",
            help="Model prediction or target variable.",
            format="%.3f",
            min_value=0,
            max_value=1,
        ),
    }
)

# --- NEW SECTION: Daily Price Charts ---
st.header("Daily Price Charts for Sample Tickers")

# Get unique tickers from the dataframe
all_tickers = data_frame['TICKER'].unique().tolist()

# Select up to the first 6 tickers to plot (no price filtering)
tickers_to_plot = all_tickers[:6]

# Only create the columns and charts if we found any tickers
if tickers_to_plot:
    # Create a column for each ticker we are plotting
    cols = st.columns(len(tickers_to_plot))

    # Iterate through the columns and the filtered tickers to create a chart in each column
    for i, ticker in enumerate(tickers_to_plot):
        with cols[i]:
            st.subheader(ticker)
            
            # Filter the dataframe for the current ticker
            ticker_data = data_frame[data_frame["TICKER"] == ticker]
            
            # Calculate the min and max price for *this specific ticker* to set a dynamic y-axis
            min_price = ticker_data['PRICE'].min()
            max_price = ticker_data['PRICE'].max()
            
            # Use Altair to create a chart with a custom y-axis and Snowflake blue line
            chart = alt.Chart(ticker_data).mark_line(color='#00B2EE').encode( # Snowflake blue color added here
                x=alt.X('DATE', title='Date'),
                y=alt.Y('PRICE', title='Price', scale=alt.Scale(domain=[min_price - (min_price*0.05), max_price + (max_price*0.05)])), # Dynamic domain based on min/max price
                tooltip=['DATE', 'PRICE']
            ).interactive()
            
            # Display the Altair chart
            st.altair_chart(chart, use_container_width=True)
else:
    st.info("No tickers found in this data sample to plot.")

# Data Science Agent: Financial ML Model

# Prompt 1 - 

Help me construct features with returns, eg. the last 1 day return using close price, return from **t-4** to **t-1**, return from **t-9** to **t-5**, return from **t-20** to **t-11**, and return from **t-62** to **t-21**. I also want to construct my predictive variable, which should be the future return from **t+2** to **t+6**. Please also help take the log across all return variables, including the previously constructed feature and this predictive variable. Please keep as panel data where ticker is a column.

In [ ]:
# Generated by Snowflake Copilot
from snowflake.snowpark.context import get_active_session
import pandas as pd
import numpy as np

# Get session
session = get_active_session()

# Get the data with necessary columns
data = session.table('SWT_LONDON_2025.FIN_SERV.FSI_DATA') \
    .select(['TICKER', 'DATE', 'R_1', 'R_5_1', 'R_10_5', 'R_21_10', 'R_63_21',
             'RETURN_LEAD_2', 'RETURN_LEAD_3', 'RETURN_LEAD_4', 'RETURN_LEAD_5']) \
    .to_pandas()

# Calculate future return (t+2 to t+5)
data['FUTURE_RETURN_2_5'] = (1 + data['RETURN_LEAD_2']) * (1 + data['RETURN_LEAD_3']) * \
                           (1 + data['RETURN_LEAD_4']) * (1 + data['RETURN_LEAD_5']) - 1

# Apply log transformation to all return columns
return_cols = ['R_1', 'R_5_1', 'R_10_5', 'R_21_10', 'R_63_21', 'FUTURE_RETURN_2_5']
for col in return_cols:
    data[f'LOG_{col}'] = np.log(1 + data[col])

# Select final columns
final_columns = ['TICKER', 'DATE'] + [f'LOG_{col}' for col in return_cols]
final_data = data[final_columns]

# Sort by ticker and date
final_data = final_data.sort_values(['TICKER', 'DATE'])

print(f"Final dataset shape: {final_data.shape}")
print("\nFirst few rows of the processed data:")
print(final_data.head())

# Prompt 2

With those features, I want to train a predictive **lightGBM** model with **L2 metric**. Please do a walk forward training on a quarterly basis. For each test quarter:
- **Train** on all quarters \< (Q-2)
- **Validate** on (Q-2, Q-1)
- **Test** on **Q**
- Enforce strict cutoffs so rows needing returns beyond the split end are dropped (no look-ahead).


### The Result

In [ ]:
# Generated by Snowflake Copilot
import pandas as pd
import numpy as np
from snowflake.snowpark.context import get_active_session
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Get session and data
session = get_active_session()
data = session.table('SWT_LONDON_2025.FIN_SERV.FSI_DATA') \
    .select(['TICKER', 'DATE', 'R_1', 'R_5_1', 'R_10_5', 'R_21_10', 'R_63_21',
             'RETURN_LEAD_2', 'RETURN_LEAD_3', 'RETURN_LEAD_4', 'RETURN_LEAD_5']) \
    .to_pandas()

# Calculate future return and log transform returns
data['FUTURE_RETURN_2_5'] = (1 + data['RETURN_LEAD_2']) * (1 + data['RETURN_LEAD_3']) * \
                           (1 + data['RETURN_LEAD_4']) * (1 + data['RETURN_LEAD_5']) - 1

# Log transform all return columns
return_cols = ['R_1', 'R_5_1', 'R_10_5', 'R_21_10', 'R_63_21', 'FUTURE_RETURN_2_5']
for col in return_cols:
    data[f'LOG_{col}'] = np.log(1 + data[col])

# Create quarter information
data['DATE'] = pd.to_datetime(data['DATE'])
data['YEAR_QUARTER'] = data['DATE'].dt.to_period('Q')

# Prepare features and target
feature_cols = ['LOG_R_1', 'LOG_R_5_1', 'LOG_R_10_5', 'LOG_R_21_10', 'LOG_R_63_21']
target_col = 'LOG_FUTURE_RETURN_2_5'

# Get all quarters from 2020Q1 onwards
all_quarters = sorted([q for q in data['YEAR_QUARTER'].unique() if q >= pd.Period('2020Q1', freq='Q')])

# Initialize results storage
results = []

# Walk-forward testing with improved model parameters
for test_quarter in all_quarters:
    # Define validation quarters (Q-2, Q-1)
    val_quarters = [test_quarter - 2, test_quarter - 1]
    
    # Split the data
    train_data = data[data['YEAR_QUARTER'] < val_quarters[0]]
    val_data = data[data['YEAR_QUARTER'].isin(val_quarters)]
    test_data = data[data['YEAR_QUARTER'] == test_quarter]
    
    # Create datasets
    X_train = train_data[feature_cols]
    y_train = train_data[target_col]
    X_val = val_data[feature_cols]
    y_val = val_data[target_col]
    X_test = test_data[feature_cols]
    y_test = test_data[target_col]
    
    # Train model with improved parameters
    model = lgb.LGBMRegressor(
        objective='regression',
        metric='l2',
        n_estimators=200,
        learning_rate=0.05,
        num_leaves=31,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    
    # Train with validation set
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)])
    
    # Make predictions
    test_pred = model.predict(X_test)
    
    # Calculate metrics
    mse = mean_squared_error(y_test, test_pred)
    r2 = 1 - np.sum((y_test - test_pred) ** 2) / np.sum((y_test - np.mean(y_test)) ** 2)
    
    # Store results
    results.append({
        'quarter': test_quarter,
        'train_size': len(X_train),
        'val_size': len(X_val),
        'test_size': len(X_test),
        'mse': mse,
        'r2': r2
    })

# Print results
print("Walk-forward testing results:")
for r in results:
    print(f"\nQuarter: {r['quarter']}")
    print(f"Train size: {r['train_size']}, Val size: {r['val_size']}, Test size: {r['test_size']}")
    print(f"MSE: {r['mse']:.6f}")
    print(f"R2: {r['r2']:.6f}")

# Calculate and print average metrics
avg_mse = np.mean([r['mse'] for r in results])
avg_r2 = np.mean([r['r2'] for r in results])
print(f"\nAverage MSE across all quarters: {avg_mse:.6f}")
print(f"Average R2 across all quarters: {avg_r2:.6f}")

# Save predictions for the most recent quarter
last_quarter = all_quarters[-1]
last_quarter_data = data[data['YEAR_QUARTER'] == last_quarter].copy()
X_last = last_quarter_data[feature_cols]
last_quarter_data['predicted_return'] = model.predict(X_last)
final_predictions = last_quarter_data[['TICKER', 'DATE', target_col, 'predicted_return']]
print("\nSample of final predictions for the last quarter:")
print(final_predictions.head())

## Prompt 3

Let's test if the strategy can work, starting 2021, for each portfolio construction, generate forecasts on **Tuesdays** in the test quarter.
- At **Wednesday close**, go **long top-5** and **short bottom-5** by predicted return (equal weight).
- Hold through **Thu → next Wed** (exactly the `t+2..t+6` window used for training).
- **Transaction cost**: 3.0 bps one-way applied via weekly turnover.
Show Strategy metrics:
- Information Ratio (before/after costs)
- Max drawdown
- At the end plot the equity curve on before & after cost.

In [ ]:
# Generated by Snowflake Copilot
import pandas as pd
import numpy as np
from snowflake.snowpark.context import get_active_session
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Get session and data
session = get_active_session()
data = session.table('SWT_LONDON_2025.FIN_SERV.FSI_DATA') \
    .select(['TICKER', 'DATE', 'R_1', 'R_5_1', 'R_10_5', 'R_21_10', 'R_63_21',
             'RETURN_LEAD_2', 'RETURN_LEAD_3', 'RETURN_LEAD_4', 'RETURN_LEAD_5']) \
    .to_pandas()

# Calculate future return and log transform
data['FUTURE_RETURN_2_5'] = (1 + data['RETURN_LEAD_2']) * (1 + data['RETURN_LEAD_3']) * \
                           (1 + data['RETURN_LEAD_4']) * (1 + data['RETURN_LEAD_5']) - 1

return_cols = ['R_1', 'R_5_1', 'R_10_5', 'R_21_10', 'R_63_21', 'FUTURE_RETURN_2_5']
for col in return_cols:
    data[f'LOG_{col}'] = np.log(1 + data[col])

# Create quarter and day information
data['DATE'] = pd.to_datetime(data['DATE'])
data['YEAR_QUARTER'] = data['DATE'].dt.to_period('Q')
data['DAY_OF_WEEK'] = data['DATE'].dt.day_name()

# Prepare features and target
feature_cols = ['LOG_R_1', 'LOG_R_5_1', 'LOG_R_10_5', 'LOG_R_21_10', 'LOG_R_63_21']
target_col = 'LOG_FUTURE_RETURN_2_5'

# Get quarters from 2021Q1 onwards for strategy testing
strategy_quarters = sorted([q for q in data['YEAR_QUARTER'].unique() if q >= pd.Period('2021Q1', freq='Q')])

# Initialize storage for all predictions
all_predictions = []

# Walk-forward testing starting from 2021Q1
for test_quarter in strategy_quarters:
    # Define validation quarters (Q-2, Q-1)
    val_quarters = [test_quarter - 2, test_quarter - 1]
    
    # Split the data - train on all quarters < (Q-2)
    train_data = data[data['YEAR_QUARTER'] < val_quarters[0]]
    val_data = data[data['YEAR_QUARTER'].isin(val_quarters)]
    test_data = data[data['YEAR_QUARTER'] == test_quarter]
    
    # Skip if insufficient training data
    if len(train_data) < 1000:
        continue
    
    # Prepare training data
    X_train = train_data[feature_cols].dropna()
    y_train = train_data[target_col].loc[X_train.index]
    X_val = val_data[feature_cols].dropna()
    y_val = val_data[target_col].loc[X_val.index]
    
    # Train LightGBM model
    model = lgb.LGBMRegressor(
        objective='regression',
        metric='l2',
        n_estimators=200,
        learning_rate=0.05,
        num_leaves=31,
        min_child_samples=20,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        verbose=-1
    )
    
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)])
    
    # Get Tuesdays in the test quarter for prediction
    test_tuesdays = test_data[test_data['DAY_OF_WEEK'] == 'Tuesday'].copy()
    
    if len(test_tuesdays) > 0:
        # Generate predictions for Tuesdays
        X_test_tuesday = test_tuesdays[feature_cols].dropna()
        if len(X_test_tuesday) > 0:
            predictions = model.predict(X_test_tuesday)
            test_tuesdays_clean = test_tuesdays.loc[X_test_tuesday.index].copy()
            test_tuesdays_clean['predicted_return'] = predictions
            test_tuesdays_clean['test_quarter'] = test_quarter
            all_predictions.append(test_tuesdays_clean)

# Combine all predictions
predictions_df = pd.concat(all_predictions, ignore_index=True)

# Strategy implementation
def calculate_strategy_returns(predictions_df):
    strategy_results = []
    
    # Group by date (Tuesday predictions)
    for date, group in predictions_df.groupby('DATE'):
        # Rank by predicted return
        group_sorted = group.sort_values('predicted_return', ascending=False)
        
        # Select top 5 (long) and bottom 5 (short)
        if len(group_sorted) >= 10:
            long_positions = group_sorted.head(5).copy()
            short_positions = group_sorted.tail(5).copy()
            
            # Calculate portfolio return (equal weight)
            long_return = long_positions['LOG_FUTURE_RETURN_2_5'].mean()
            short_return = short_positions['LOG_FUTURE_RETURN_2_5'].mean()
            
            # Long-short strategy return
            strategy_return = long_return - short_return
            
            # Store results
            strategy_results.append({
                'date': date,
                'long_return': long_return,
                'short_return': short_return,
                'strategy_return': strategy_return,
                'num_stocks': len(group_sorted),
                'long_tickers': list(long_positions['TICKER']),
                'short_tickers': list(short_positions['TICKER'])
            })
    
    return pd.DataFrame(strategy_results)

# Calculate strategy returns
strategy_df = calculate_strategy_returns(predictions_df)

# Calculate transaction costs based on turnover
def calculate_turnover_and_costs(strategy_df):
    strategy_df = strategy_df.copy()
    strategy_df['turnover'] = 0.0
    strategy_df['transaction_cost'] = 0.0
    
    prev_long = set()
    prev_short = set()
    
    for i, row in strategy_df.iterrows():
        current_long = set(row['long_tickers'])
        current_short = set(row['short_tickers'])
        
        if i > 0:
            # Calculate turnover as fraction of positions that changed
            long_changes = len(current_long.symmetric_difference(prev_long))
            short_changes = len(current_short.symmetric_difference(prev_short))
            total_changes = long_changes + short_changes
            total_positions = 10  # 5 long + 5 short
            
            turnover = total_changes / total_positions
            # Transaction cost: 3.0 bps one-way on turnover
            transaction_cost = turnover * 0.0003  # 3.0 bps = 0.0003
            
            strategy_df.loc[i, 'turnover'] = turnover
            strategy_df.loc[i, 'transaction_cost'] = transaction_cost
        
        prev_long = current_long
        prev_short = current_short
    
    return strategy_df

# Calculate costs
strategy_df = calculate_turnover_and_costs(strategy_df)

# Calculate net returns after transaction costs
strategy_df['net_return'] = strategy_df['strategy_return'] - strategy_df['transaction_cost']

# Calculate cumulative returns and strategy metrics
strategy_df['cumulative_gross'] = (1 + strategy_df['strategy_return']).cumprod()
strategy_df['cumulative_net'] = (1 + strategy_df['net_return']).cumprod()

# Calculate Information Ratio (annualized)
weeks_per_year = 52
gross_annual_return = strategy_df['strategy_return'].mean() * weeks_per_year
net_annual_return = strategy_df['net_return'].mean() * weeks_per_year
gross_annual_vol = strategy_df['strategy_return'].std() * np.sqrt(weeks_per_year)
net_annual_vol = strategy_df['net_return'].std() * np.sqrt(weeks_per_year)

ir_gross = gross_annual_return / gross_annual_vol
ir_net = net_annual_return / net_annual_vol

# Calculate Maximum Drawdown
def calculate_max_drawdown(cumulative_returns):
    peak = cumulative_returns.expanding().max()
    drawdown = (cumulative_returns - peak) / peak
    return drawdown.min()

max_dd_gross = calculate_max_drawdown(strategy_df['cumulative_gross'])
max_dd_net = calculate_max_drawdown(strategy_df['cumulative_net'])

# Print strategy metrics
print("=== STRATEGY PERFORMANCE METRICS ===")
print(f"\nGross Performance (Before Costs):")
print(f"  Annual Return: {gross_annual_return:.2%}")
print(f"  Annual Volatility: {gross_annual_vol:.2%}")
print(f"  Information Ratio: {ir_gross:.3f}")
print(f"  Maximum Drawdown: {max_dd_gross:.2%}")

print(f"\nNet Performance (After Costs):")
print(f"  Annual Return: {net_annual_return:.2%}")
print(f"  Annual Volatility: {net_annual_vol:.2%}")
print(f"  Information Ratio: {ir_net:.3f}")
print(f"  Maximum Drawdown: {max_dd_net:.2%}")

print(f"\nTransaction Cost Impact:")
print(f"  Cost Drag: {(gross_annual_return - net_annual_return):.2%} annually")
print(f"  Average Weekly Turnover: {strategy_df['turnover'].mean():.1%}")

# Final performance summary
total_gross_return = strategy_df['cumulative_gross'].iloc[-1] - 1
total_net_return = strategy_df['cumulative_net'].iloc[-1] - 1
print(f"\nTotal Strategy Performance ({strategy_df['date'].min().strftime('%Y-%m-%d')} to {strategy_df['date'].max().strftime('%Y-%m-%d')}):")
print(f"  Gross Total Return: {total_gross_return:.2%}")
print(f"  Net Total Return: {total_net_return:.2%}")

# Create equity curve plot
plt.figure(figsize=(12, 8))
plt.plot(strategy_df['date'], strategy_df['cumulative_gross'], label='Gross Returns (Before Costs)', linewidth=2)
plt.plot(strategy_df['date'], strategy_df['cumulative_net'], label='Net Returns (After Costs)', linewidth=2)
plt.axhline(y=1, color='black', linestyle='--', alpha=0.5, label='Breakeven')

plt.title('Long-Short Strategy Equity Curve (2021-2025)', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative Return (Base = 1)', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()

# Add performance annotations
final_gross = strategy_df['cumulative_gross'].iloc[-1]
final_net = strategy_df['cumulative_net'].iloc[-1]
plt.text(0.02, 0.98, f'Final Gross: {final_gross:.3f} (+{(final_gross-1)*100:.1f}%)', 
         transform=plt.gca().transAxes, fontsize=10, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
plt.text(0.02, 0.90, f'Final Net: {final_net:.3f} (+{(final_net-1)*100:.1f}%)', 
         transform=plt.gca().transAxes, fontsize=10, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.8))

plt.show()

print("Strategy implementation completed successfully!")
print(f"Total trading weeks: {len(strategy_df)}")
print(f"Strategy period: {strategy_df['date'].min().strftime('%Y-%m-%d')} to {strategy_df['date'].max().strftime('%Y-%m-%d')}")

## Prompt 4

Register only the final model in Snowflake Registry with the following options:

- model name "STOCK_RETURN_PREDICTOR_GBM"
- sample input of 100 rows
- target_platforms=["WAREHOUSE"]
- options={
    "relax_version": False,
    "target_methods": ["predict"],
    "method_options": {
        "predict": {"case_sensitive": True}
    }
}

In [ ]:
# Generated by Snowflake Copilot
from snowflake.snowpark.context import get_active_session
import pandas as pd
import numpy as np
import lightgbm as lgb
from snowflake.ml.registry import Registry

session = get_active_session()

data = session.table('SWT_LONDON_2025.FIN_SERV.FSI_DATA') \
    .select(['TICKER', 'DATE', 'R_1', 'R_5_1', 'R_10_5', 'R_21_10', 'R_63_21',
             'RETURN_LEAD_2', 'RETURN_LEAD_3', 'RETURN_LEAD_4', 'RETURN_LEAD_5']) \
    .to_pandas()

data['FUTURE_RETURN_2_5'] = (1 + data['RETURN_LEAD_2']) * (1 + data['RETURN_LEAD_3']) * \
                           (1 + data['RETURN_LEAD_4']) * (1 + data['RETURN_LEAD_5']) - 1

return_cols = ['R_1', 'R_5_1', 'R_10_5', 'R_21_10', 'R_63_21', 'FUTURE_RETURN_2_5']
for col in return_cols:
    data[f'LOG_{col}'] = np.log(1 + data[col])

data['DATE'] = pd.to_datetime(data['DATE'])
data['YEAR_QUARTER'] = data['DATE'].dt.to_period('Q')

feature_cols = ['LOG_R_1', 'LOG_R_5_1', 'LOG_R_10_5', 'LOG_R_21_10', 'LOG_R_63_21']
target_col = 'LOG_FUTURE_RETURN_2_5'

training_data = data[data['YEAR_QUARTER'] < pd.Period('2025Q4', freq='Q')].copy()
training_data_clean = training_data.dropna(subset=feature_cols + [target_col])

X_train = training_data_clean[feature_cols]
y_train = training_data_clean[target_col]

final_model = lgb.LGBMRegressor(
    objective='regression',
    metric='l2',
    n_estimators=200,
    learning_rate=0.05,
    num_leaves=31,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbose=-1
)

final_model.fit(X_train, y_train)

sample_input = X_train.head(100)

registry = Registry(session=session)

model_ref = registry.log_model(
    model=final_model,
    model_name="STOCK_RETURN_PREDICTOR_GBM",
    sample_input_data=sample_input,
    target_platforms=["WAREHOUSE"],
    options={
        "relax_version": False,
        "target_methods": ["predict"],
        "method_options": {
            "predict": {"case_sensitive": True}
        }
    }
)

print(f"Model registered successfully!")
print(f"Model name: {model_ref.model_name}")
print(f"Model version: {model_ref.version_name}")
print(f"Sample input shape: {sample_input.shape}")
print(f"Feature columns used: {list(sample_input.columns)}")